<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 6: Использование Advantage Actor Critic (A2C) для моделирования робототехники с помощью PyBullet и Panda-Gym 🤖

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/thumbnail.png"  alt="Thumbnail" width="50%"/>

В этом блокноте вы научитесь использовать A2C с двумя наборами робототехнических сред PyBullet и Panda-Gym.

С [PyBullet](https://github.com/bulletphysics/bullet3), мы собираемся **обучить робота двигаться**:
- `AntBulletEnv-v0` 🕸️ Точнее, паук (они говорят Муравей, но да ладно... это паук 😆 ) 🕸️

Затем, с [Panda-Gym](https://github.com/qgallouedec/panda-gym), мы собираемся **обучить роботизированную руку** (Франка Эмика Панда робот) выполнять задание:
- `Reach`: робот должен поместить свой конечный манипулятор в целевое положение.

После этого вы сможете **тренироваться в других средах робототехники**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/environments.gif" alt="Robotics environments" width="50%"/>

### 🎮 Окружающая среда: 

- [PyBullet](https://github.com/bulletphysics/bullet3)
- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

###📚 RL-Библиотека: 

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

## Цели этого блокнота 🏆

В конце этого блокнота вы:

- Научитесь использовать библиотеки сред **PyBullet** и **Panda-Gym**.
-  Научитесь **обучать роботов, используя A2C**.
- Поймите, почему **нам нужно нормализовать входные данные**.
- Будьте в состоянии **отправить своего обученного агента и код в центр** с хорошим видеоповтором и оценочным баллом 🔥.

## Этот блокнот взят из курса обучения с глубоким подкреплением
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="50%"/>

🔲 📚 [Метод "актер-критик" Блок 6](https://huggingface.co/deep-rl-course/unit6/introduction) 🤗  

# Давайте обучим наших первых роботов 🤖

Чтобы подтвердить это практическое руководство для [процесса сертификации](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process),  вам нужно отправить две ваши обученные модели в концентратор и получить следующие результаты:

- `AntBulletEnv-v0` получив результат >= 650.
- `PandaReachDense-v2` получив результат >= -3.5.

Чтобы найти свой результат, перейдите к [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) и найдите свою модель, **the result = mean_reward - std of reward**

Если вы не нашли свою модель, **перейдите в нижнюю часть страницы и нажмите на кнопку обновить**

Для получения дополнительной информации о процессе сертификации ознакомьтесь с этим разделом 👉 https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process

## Создайте виртуальный дисплей 🔽

Во время записи нам нужно будет сгенерировать видео-повтор. Для этого с помощью colab ** нам нужен виртуальный экран, чтобы иметь возможность отображать среду ** (и, таким образом, записывать кадры).

Следовательно, в следующей ячейке будут установлены библиотеки, а также создан и запущен виртуальный экран 🖥

In [ ]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### Установка зависимостей 🔽
Первым шагом является установка зависимостей, мы установим несколько из них:

- `pybullet`: Содержит среду обитания шагающих роботов.
- `panda-gym`: Содержит среду роботизированной руки.
- `стабильные базовые линии 3 [дополнительно]`: Библиотека обучения с глубоким подкреплением SB3.
- `hugging face_sb3`: Дополнительный код для Stable-baselines 3 для загрузки моделей из Hugging Face 🤗 Hub.
- `hugging face_hub`: Библиотека, позволяющая любому пользователю работать с репозиториями хаба.

In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit6/requirements-unit6.txt

## Импортируйте пакеты 📦

In [ ]:
import pybullet_envs
import panda_gym
import gym

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

## Окружающая среда №1: AntBulletEnv-v0 🕸


### Создайте AntBulletEnv-v0
#### Окружающая среда 🎮
В этой среде агент должен правильно использовать свои различные суставы, чтобы правильно ходить.
Вы можете найти подробное объяснение этой среды здесь: https://hackmd.io/@jeffreymo/SJJrSJh5_#PyBullet

In [ ]:
env_id = "AntBulletEnv-v0"
# Создайте окружающую среду
env = gym.make(env_id)

# Получите пространство состояний и пространство действий
s_size = env.observation_space.shape[0]
a_size = env.action_space

In [ ]:
print("_____ПРОСТРАНСТВО НАБЛЮДЕНИЙ_____ \n")
print("Размер пространства состояний: ", s_size)
print("Выборочное наблюдение", env.observation_space.sample()) # Получите случайное наблюдение

Пространство наблюдения (от [Jeffrey Y Mo](https://hackmd.io/@jeffreymo/SJJrSJh5_#PyBullet)):

Разница в том, что наше пространство наблюдения равно 28, а не 29.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/obs_space.png" alt="PyBullet Ant Obs space" width="70%"/>

In [ ]:
print("\n _____ПРОСТРАНСТВО ДЕЙСТВИЙ_____ \n")
print("Размер пространства действий: ", a_size)
print("Пример действия из пространства", env.action_space.sample()) # Получите случайное действие

Пространство действий (от [Jeffrey Y Mo](https://hackmd.io/@jeffreymo/SJJrSJh5_#PyBullet)):

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit8/action_space.png" alt="PyBullet Ant Obs space" width="70%"/>

### Нормализуйте наблюдение и вознаграждения